# <font color='blue'> ONNX - Open Neural Network Exchange</font>


<img src="https://www.learnopencv.com/wp-content/uploads/2020/09/c3-w15-onnx.png" height="500">

---

ONNX, as the name says, is a platform for exchanging the formats `(.pth(Pytorch), .pb(TensorFlow), etc.)` of Neural Networks.

**Why do we need ONNX?**

Often, the training environment (Python) is different from the production environment (Java, C#, etc.), and these models need to be deployed.

Since we have trained the models in either any of the Deep Learning frameworks,  we somehow want to port this model for serving. And this is where ONNX appears to be pretty handy.


Sometimes, we might also need to switch the model between frameworks ie, from `Pytorch to MXNET` or from `Keras to Pytorch`, etc., and ONNX can help us here too.

We also need to note that every framework has its way of storing the weights ie, Keras saves the model-weights as a `.h5` file, Pytorch just stores the weights in a `.pth` file, so we need a way to store the weights such that all the frameworks can recognize it.

## <font color='blue'>Installations</font>

Before we start, we need to install the required dependencies. So let's start with installation.

In [1]:
!pip install onnx # to load the onnx model
!pip install onnxruntime # to load the onnx model
!pip install onnx2keras # to convert an onnx-model to keras-model.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for onnx2keras: filename=onnx2keras-0.0.24-py3-none-any.whl size=24576 sha256=123aa01baaf43718990efd46a0af5738e19c95499bcf8f3b57cda0d44aab26d2
  Stored in directory: /root/.cache/pip/wheels/3e/2c/e2/eb2ef49a50cab50b6d29557da4fefe10af3e9284412d7ef4f9
Successfully built onnx2keras


## <font color='green'>1. PyTorch to ONNX</font>

PyTorch provides us a very easy-to-use function to convert a PyTorch model to an ONNX format ending with an extension `.onnx`.

Let's go ahead and see how we can do it.

In [2]:
# We will need to import the necessary libraries
import onnx
import onnxruntime
import torch
from torchvision import models
import numpy as np

In [3]:
# Pick a model from torchvision to port it to ONNX.
# We shall use the `resnet18()` to port it to ONNX.
resnet = models.resnet18()

# Place the model in `evaluation-mode`
resnet.eval()
# Create a random input and get the output
ip = torch.randn(1,3, 224, 224)
with torch.no_grad():
    op = resnet(ip)

# Lets take the output-summation to see if we get the same output inferring through ONNX too
print("The output from the PyTorch model: ",op.sum())


The output from the PyTorch model:  tensor(-48.9275)


In [4]:
# We shall convert the Pytorch model to ONNX using the function below.
torch.onnx.export(resnet, ip, "resnet.onnx" )

/tmp/ipython-input-2942324263.py:2: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(resnet, ip, "resnet.onnx" )


In [26]:
# Export the model to ONNX format
torch.onnx.export(
    resnet,                     # The model to be exported
    ip,                         # The sample input tensor
    "model.onnx",               # The output file name
    export_params=True,         # Store the trained parameter weights inside the model file
    opset_version=12,           # The ONNX version to export the model to
    do_constant_folding=True,   # Whether to execute constant folding for optimization
    input_names=['input'],      # The model's input names
    output_names=['output'],    # The model's output names
)

/tmp/ipython-input-668913669.py:2: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


##  <font color='green'>2. Inferring in ONNX</font>

In [5]:
## We need to create a session for inference when running an onnx model.
session = onnxruntime.InferenceSession("resnet.onnx")

# We also need to get the input_name.
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

## Run the session be specifying our input at the node `input_name` and specifying that
## we need to break the graph at the node `output_name`
result = session.run([output_name], {input_name: ip.numpy()})

In [6]:
## The `result` will be a list since we passed the outputs in the form of a list.
print("The output from the ONNX model: ",result[0].sum())

The output from the ONNX model:  -48.927475


## <font color='green'>3. Inferring in OpenCV</font>

 Often, the production environment is constrained to only a few libraries. And we cannot afford to use the `Pytorch` library, since we need it to only infer on the input.

Hence, we need a way to load this `PyTorch` model into existing libraries, and one such library is `OpenCV`.

Although the latest version of `OpenCV` already has support to load models from `TensorFlow`,  `Caffe`, etc., we don't have an API for loading `PyTorch` models. And, guess what, ONNX comes to our rescue here.

In the following cell, we shall see how we can load the converted PyTorch model in OpenCV.


In [7]:
# import the opencv library
import cv2

# use the `readNetFromONNX` API to load the saved-onnx-model.
lenet_onnx = cv2.dnn.readNetFromONNX("resnet.onnx")

In [8]:
# we shall set the input to the network
lenet_onnx.setInput(ip.numpy())

# get the output of model by calling the `forward()` method.
onnx_op = lenet_onnx.forward()

# let's verify the sum of this output with the previous results.
print("The output from the ONNX model when loaded via OpenCV: ", onnx_op.sum())

The output from the ONNX model when loaded via OpenCV:  -48.92751


## <font color='green'>4. ONNX to Keras and Inferring in Keras</font>

Sometimes, the production environment requires us to have a Tensorflow-based environment.  

In that case, too, we can use the module `onnx2keras` to convert from `ONNX` to `Keras` and then save this model to the Keras format `.h5`

In [12]:
!pip install --upgrade onnx2keras

In [13]:
# importing the onnx_to_keras function
from onnx2keras import onnx_to_keras

In [27]:
# load the onnx-model converted from pytorch.
onnx_model = onnx.load("model.onnx")

In [28]:
k_model = onnx_to_keras(onnx_model, ['input'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


ValueError: Argument `name` must be a string and cannot contain character `/`. Received: name=/conv1/Conv_output_0_pad (of type <class 'str'>)

In [15]:
# we shall specify the required arguments to the function. The argument `change_ordering`
# makes the model portable to input shape (B, H, W, C).

keras_model = onnx_to_keras(onnx_model=onnx_model, input_names=[input_name], change_ordering=True)
# this `keras_model` is now the Resnet-graph built in Keras.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


ValueError: Argument `name` must be a string and cannot contain character `/`. Received: name=/conv1/Conv_output_0_pad (of type <class 'str'>)

In [ ]:
# We can now use this graph to verify if we get the same results.

# We should use the `predict()` method to call the evaluation-mode.
# Also note that, the input is permuted from [B, C, H, W] to [B, H, W, C].

keras_op = keras_model.predict(ip.permute(0, 2, 3, 1).numpy())
print("Output sum when converting from ONNX to Keras: ", keras_op.sum())

Output sum when converting from ONNX to Keras:  -3.2902946
